# Importing Libraries

In [1]:
import tweepy
import webbrowser
import pandas as pd
import re
import os
import json
from emoji import  demojize
from textblob import TextBlob
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import numpy as np
from google_trans_new import google_translator
import time

# Authentication with Twitter API

In [2]:
CONSUMER_KEY = "PVqljUygPmbZXd6H6jgCXNJ1K"
CONSUMER_SECRET_KEY = "7OvPwP9dYL488fY0eUqrLpv1medx4vxSjxNV05zh89MowuUo2d"
callback_uri = 'oob'
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET_KEY, callback_uri)
redirect_url = auth.get_authorization_url()
webbrowser.open(redirect_url)
user_pin_input = input('What is the pin value? ')
auth.get_access_token(user_pin_input)

What is the pin value? 1424424


('2942085154-3P3Vs01tk9oXoysrw8MAQKUeoCd8wpRilJ954Zc',
 'YEhZgKyGbeatl3boSZOAD73WL3d8ZgiY9l5TtNg9lNvx9')

In [3]:
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Reading and Parsing Tweets

In [4]:
def cleanTweet(tweet):  # Function that cleans tweets
    
    tweet = re.sub(r'@[A-Za-z0-9]+', '', tweet)  # removing the @ symbol
    tweet = re.sub('\\n', '', tweet)  # removing '\n' 
    tweet = re.sub('https?:\/\/\S+', '', tweet)  # removing hyperlinks
    return tweet

In [5]:
def readTsvFile(fileName, country_code):  # Function that reads a TSV 
    
    df = pd.read_csv(fileName, sep='\t') # reading .tsv file as a dataframe

    df_cc = df[df['country_code'] == country_code]  # getting tweets with country_code
    df_cc = df_cc.reset_index(drop=True)  # resetting rows

    df_en = df_cc[df_cc['lang'] == "en"]  # getting en language -> English

    df_en = df_en.reset_index(drop=True)  # resetting rows
    
    return df_en

## Reading every file in dataset

The 'readCorpus()' function is used to read all TSV files from the existing covid-19 dataset for every country.

In [6]:
df_GB_list = []
df_IT_list = []
df_ES_list = []

def readCorpus(corpusPath):
    for root, dirs, files in os.walk(corpusPath):
        for name in files:
            fileName = os.path.join(root,name)
            print(fileName)
            with open(fileName, "r", encoding="utf-8") as f:
                df_GB_list.append(readTsvFile(fileName, "GB"))
                df_IT_list.append(readTsvFile(fileName, "IT"))
                df_ES_list.append(readTsvFile(fileName, "ES"))

In [7]:
readCorpus("Pre-Dataset")
readCorpus("Pre-Dataset_2")
readCorpus("Post-Dataset")
readCorpus("Post-Dataset_2")

## Concatenating every dataframe into 1 dataframe

In [109]:
# Concatenate/Merging multiple dfs into 1
df_GB = pd.concat(df_GB_list)
df_IT = pd.concat(df_IT_list)
df_ES = pd.concat(df_ES_list)

## Parsing tweets

The 'parseTweets()' function is used to parse every tweet collected from the covid-19 existing dataset. This is achieved by checking for keywords in the tweet's text, and if a keyword is found, then that tweet is considered relevant to covid-19 vaccine. This function returns a dataframe with all the tweets parsed.

In [7]:
def parseTweets(df):
    
    keywords_list = ["vaccine", "vaccinated", "jab", "injection", "astrazeneca", "moderna", "pfizer", "johnson & johnson", "j&j", "side effects", "blood clot", "thrombosis"]
    
    #translator = google_translator()  # google API
    
    tweet_id = df["tweet_id"].tolist()  # getting tweets ids

    i = 0
    tweets = []
    for tweet in tweet_id:
        try:
            tweet_text = api.get_status(tweet).text
            tweet_text = cleanTweet(tweet_text).lower()
            tweet_text = deEmojify(tweet_text)
            #print(tweet_text+'\t'+translator.translate(tweet_text,lang_tgt='en'))

            #try:
            #    tweet_text = translator.translate(tweet_text,lang_tgt='en') 
            #except json.decoder.JSONDecodeError:
            #    pass
                
            if any(x in tweet_text for x in keywords_list):
                i+=1
                tweets.append(tweet_text)
            
        except tweepy.TweepError as err:
            continue
    
    final_df = pd.DataFrame(tweets, columns =['Tweets'])
    return final_df.drop_duplicates()

In [8]:
def deEmojify(text):  # Function that removes emojis from text
    
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [112]:
df_GB_cleaned = parseTweets(df_GB)
df_GB_cleaned
#df_IT = parseTweets(df_ITc)

Rate limit reached. Sleeping for: 584
Rate limit reached. Sleeping for: 577


,Tweets
0,#covid19 jab day two today #covidvaccination #...
1,"_ireland ""the injections are dangerous biolog..."
2,1st dose vaccinations available to #doncasteri...
3,🤔i'd like to learn why isn't going fully onli...
4,"""the injections are dangerous biological agen..."
...,...
267,us arranges 277.3 million doses of covid-19 va...
268,need your first #covidvaccine and live in #don...
269,second-dose covid-19 jabs brought forward as i...
270,inching closer and closer to that sweet sweet ...


In [113]:
df_IT_cleaned = parseTweets(df_IT)
df_IT_cleaned

,Tweets
0,global vaccination effort faces fresh challeng...
1,my friends make my feel like a crimminal. due ...
2,a few countries have ordered enough #covid19 v...
3,dr. dorry segev on cbs news via #vaccineswor...
4,it’s a beautiful day to save the world ❤️ 1/2🤩...
5,"the #taiwan, #us, #japan, #uk, #australia #gct..."
6,eventually vaccine booking for my age group op...


In [114]:
df_ES_cleaned = parseTweets(df_ES)
df_ES_cleaned

,Tweets
0,vaccinated... and it feels so good 🤪i got the ...
1,pfizer covid-19 vaccine more durable then thou...
2,preliminary: a single dose of astrazeneca covi...
3,good news !#covid19 #astrazeneca #pfizer
4,#fotografiadestock #stockphotography #lab #cov...
5,#fotografiadestock #stockphotography #lab #cov...


# Searching for Tweets using Twitter API 'Search' endpoint

The following are the locations of each country where tweets relating to covid-19 where searched:

"London_GB": "51.500985,-0.126127",
"Milan_IT": "45.4642,9.1900",
"Madrid_ES": "40.415355,-3.706138"

"Rome_IT": "41.9028,12.4964",
"Zaragoza_ES": "41.651381,-0.885892"

"Birmingham_GB": "52.482326,-1.894550",

"Edinburgh_GB": "55.948661,-3.187444",
"Glasgow_GB": "55.864200,-4.251800",
"Seville_ES": "37.389100,-5.984500",
"Barcelona_ES": "41.3851,2.1734",
"Naples_IT": "40.8518,14.2681",
"Bologna_IT": "44.4949,11.3426"

"Sheffield_GB": "53.381100,-1.470100"

"Belfast_GB": "54.597300,-5.930100",
"Liverpool_GB": "53.409006,-2.986110",
"Malaga_ES": "36.7213,-4.4213",
"Valencia_ES": "39.4699,-0.3763",
"Turin_IT": "45.0703,7.6869",
"Florence_IT": "43.7696,11.2558"

- NB: 'GB' -> Great Britain, 'IT' -> Italy, 'ES' -> Spain

In [10]:
search_df_list = []

In [11]:
# Create a dictionary of locations
# Create a dictionary of search keywords


# Loop through each location
    # Loop through each search keyword
        # Search for tweets -> store in temp df
        # Append to final df

        
s = 120      
        
country_locations = {
    
    "London_GB": "51.500985,-0.126127", 
    "Milan_IT": "45.4642,9.1900", 
    "Madrid_ES": "40.415355,-3.706138",
    "Rome_IT": "41.9028,12.4964", 
    "Zaragoza_ES": "41.651381,-0.885892",
    "Birmingham_GB": "52.482326,-1.894550",
    "Edinburgh_GB": "55.948661,-3.187444", 
    "Glasgow_GB": "55.864200,-4.251800", 
    "Seville_ES": "37.389100,-5.984500", 
    "Barcelona_ES": "41.3851,2.1734", 
    "Naples_IT": "40.8518,14.2681", 
    "Bologna_IT": "44.4949,11.3426",
    "Sheffield_GB": "53.381100,-1.470100",
    "Belfast_GB": "54.597300,-5.930100", 
    "Liverpool_GB": "53.409006,-2.986110",
    "Malaga_ES": "36.7213,-4.4213", 
    "Valencia_ES": "39.4699,-0.3763", 
    "Turin_IT": "45.0703,7.6869", 
    "Florence_IT": "43.7696,11.2558"
    
}  

search_keywords = {
    "vaccine -filter:retweets", "vaccinated -filter:retweets", "covid-19 vaccine -filter:retweets", "jab -filter:retweets", "injection -filter:retweets", 
    "astrazeneca -filter:retweets", "moderna -filter:retweets", "pfizer -filter:retweets", "janssen -filter:retweets", 
    "j&j -filter:retweets", "side effects -filter:retweets", "blood clot -filter:retweets", 
    "thrombosis -filter:retweets", "covid19 covid vaccine -filter:retweets"
}

language = "en"

for key,value in country_locations.items():
    for keyword in search_keywords:
        try:
            print(f"Country: {key}\nKeyword: {keyword}\nGeocode: {value}")
            tweets = tweepy.Cursor(api.search, q=keyword, lang=language, geocode=value+",100km").items(200)
            tweets_text = [deEmojify(cleanTweet(t.text)) for t in tweets]
            
            tweets_size = len(tweets_text)
            idx = key.find('_')+1
            
            country = [key[idx:] for i in range(tweets_size)]
                    
            data = {'Tweets': tweets_text, 'Country': country}
            temp_df = pd.DataFrame(data)
            search_df_list.append(temp_df)
            
        except tweepy.TweepError as err:
            print(f"TweepError: Search again in {s} seconds")
            time.sleep(s)

Country: London_GB
Keyword: blood clot -filter:retweets
Geocode: 51.500985,-0.126127
Country: London_GB
Keyword: thrombosis -filter:retweets
Geocode: 51.500985,-0.126127
Country: London_GB
Keyword: vaccine -filter:retweets
Geocode: 51.500985,-0.126127
Country: London_GB
Keyword: janssen -filter:retweets
Geocode: 51.500985,-0.126127
Country: London_GB
Keyword: j&j -filter:retweets
Geocode: 51.500985,-0.126127
Country: London_GB
Keyword: covid19 covid vaccine -filter:retweets
Geocode: 51.500985,-0.126127
Country: London_GB
Keyword: covid-19 vaccine -filter:retweets
Geocode: 51.500985,-0.126127
Country: London_GB
Keyword: injection -filter:retweets
Geocode: 51.500985,-0.126127
Country: London_GB
Keyword: vaccinated -filter:retweets
Geocode: 51.500985,-0.126127
Country: London_GB
Keyword: astrazeneca -filter:retweets
Geocode: 51.500985,-0.126127
Country: London_GB
Keyword: jab -filter:retweets
Geocode: 51.500985,-0.126127
Country: London_GB
Keyword: side effects -filter:retweets
Geocode: 5

Rate limit reached. Sleeping for: 777


Country: Milan_IT
Keyword: blood clot -filter:retweets
Geocode: 45.4642,9.1900
Country: Milan_IT
Keyword: thrombosis -filter:retweets
Geocode: 45.4642,9.1900
Country: Milan_IT
Keyword: vaccine -filter:retweets
Geocode: 45.4642,9.1900
Country: Milan_IT
Keyword: janssen -filter:retweets
Geocode: 45.4642,9.1900
Country: Milan_IT
Keyword: j&j -filter:retweets
Geocode: 45.4642,9.1900
Country: Milan_IT
Keyword: covid19 covid vaccine -filter:retweets
Geocode: 45.4642,9.1900
Country: Milan_IT
Keyword: covid-19 vaccine -filter:retweets
Geocode: 45.4642,9.1900
Country: Milan_IT
Keyword: injection -filter:retweets
Geocode: 45.4642,9.1900
Country: Milan_IT
Keyword: vaccinated -filter:retweets
Geocode: 45.4642,9.1900
Country: Milan_IT
Keyword: astrazeneca -filter:retweets
Geocode: 45.4642,9.1900
Country: Milan_IT
Keyword: jab -filter:retweets
Geocode: 45.4642,9.1900
Country: Milan_IT
Keyword: side effects -filter:retweets
Geocode: 45.4642,9.1900
Country: Milan_IT
Keyword: pfizer -filter:retweets
Ge

Rate limit reached. Sleeping for: 792


Country: Zaragoza_ES
Keyword: covid-19 vaccine -filter:retweets
Geocode: 41.651381,-0.885892
Country: Zaragoza_ES
Keyword: injection -filter:retweets
Geocode: 41.651381,-0.885892
Country: Zaragoza_ES
Keyword: vaccinated -filter:retweets
Geocode: 41.651381,-0.885892
Country: Zaragoza_ES
Keyword: astrazeneca -filter:retweets
Geocode: 41.651381,-0.885892
Country: Zaragoza_ES
Keyword: jab -filter:retweets
Geocode: 41.651381,-0.885892
Country: Zaragoza_ES
Keyword: side effects -filter:retweets
Geocode: 41.651381,-0.885892
Country: Zaragoza_ES
Keyword: pfizer -filter:retweets
Geocode: 41.651381,-0.885892
Country: Zaragoza_ES
Keyword: moderna -filter:retweets
Geocode: 41.651381,-0.885892
Country: Birmingham_GB
Keyword: blood clot -filter:retweets
Geocode: 52.482326,-1.894550
Country: Birmingham_GB
Keyword: thrombosis -filter:retweets
Geocode: 52.482326,-1.894550
Country: Birmingham_GB
Keyword: vaccine -filter:retweets
Geocode: 52.482326,-1.894550
Country: Birmingham_GB
Keyword: janssen -filte

...

Country: Sheffield_GB
Keyword: moderna -filter:retweets
Geocode: 53.381100,-1.470100
Country: Belfast_GB
Keyword: blood clot -filter:retweets
Geocode: 54.597300,-5.930100
Country: Belfast_GB
Keyword: thrombosis -filter:retweets
Geocode: 54.597300,-5.930100
Country: Belfast_GB
Keyword: vaccine -filter:retweets
Geocode: 54.597300,-5.930100
Country: Belfast_GB
Keyword: janssen -filter:retweets
Geocode: 54.597300,-5.930100
Country: Belfast_GB
Keyword: j&j -filter:retweets
Geocode: 54.597300,-5.930100
Country: Belfast_GB
Keyword: covid19 covid vaccine -filter:retweets
Geocode: 54.597300,-5.930100
Country: Belfast_GB
Keyword: covid-19 vaccine -filter:retweets
Geocode: 54.597300,-5.930100
Country: Belfast_GB
Keyword: injection -filter:retweets
Geocode: 54.597300,-5.930100
Country: Belfast_GB
Keyword: vaccinated -filter:retweets
Geocode: 54.597300,-5.930100
Country: Belfast_GB
Keyword: astrazeneca -filter:retweets
Geocode: 54.597300,-5.930100
Country: Belfast_GB
Keyword: jab -filter:retweets
G

Rate limit reached. Sleeping for: 775


Country: Liverpool_GB
Keyword: vaccinated -filter:retweets
Geocode: 53.409006,-2.986110
Country: Liverpool_GB
Keyword: astrazeneca -filter:retweets
Geocode: 53.409006,-2.986110
Country: Liverpool_GB
Keyword: jab -filter:retweets
Geocode: 53.409006,-2.986110
Country: Liverpool_GB
Keyword: side effects -filter:retweets
Geocode: 53.409006,-2.986110
Country: Liverpool_GB
Keyword: pfizer -filter:retweets
Geocode: 53.409006,-2.986110
Country: Liverpool_GB
Keyword: moderna -filter:retweets
Geocode: 53.409006,-2.986110
Country: Malaga_ES
Keyword: blood clot -filter:retweets
Geocode: 36.7213,-4.4213
Country: Malaga_ES
Keyword: thrombosis -filter:retweets
Geocode: 36.7213,-4.4213
Country: Malaga_ES
Keyword: vaccine -filter:retweets
Geocode: 36.7213,-4.4213
Country: Malaga_ES
Keyword: janssen -filter:retweets
Geocode: 36.7213,-4.4213
Country: Malaga_ES
Keyword: j&j -filter:retweets
Geocode: 36.7213,-4.4213
Country: Malaga_ES
Keyword: covid19 covid vaccine -filter:retweets
Geocode: 36.7213,-4.4213

Rate limit reached. Sleeping for: 786


Country: Turin_IT
Keyword: covid19 covid vaccine -filter:retweets
Geocode: 45.0703,7.6869
Country: Turin_IT
Keyword: covid-19 vaccine -filter:retweets
Geocode: 45.0703,7.6869
Country: Turin_IT
Keyword: injection -filter:retweets
Geocode: 45.0703,7.6869
Country: Turin_IT
Keyword: vaccinated -filter:retweets
Geocode: 45.0703,7.6869
Country: Turin_IT
Keyword: astrazeneca -filter:retweets
Geocode: 45.0703,7.6869
Country: Turin_IT
Keyword: jab -filter:retweets
Geocode: 45.0703,7.6869
Country: Turin_IT
Keyword: side effects -filter:retweets
Geocode: 45.0703,7.6869
Country: Turin_IT
Keyword: pfizer -filter:retweets
Geocode: 45.0703,7.6869
Country: Turin_IT
Keyword: moderna -filter:retweets
Geocode: 45.0703,7.6869
Country: Florence_IT
Keyword: blood clot -filter:retweets
Geocode: 43.7696,11.2558
Country: Florence_IT
Keyword: thrombosis -filter:retweets
Geocode: 43.7696,11.2558
Country: Florence_IT
Keyword: vaccine -filter:retweets
Geocode: 43.7696,11.2558
Country: Florence_IT
Keyword: janssen 

In [12]:
search_df_list

[                                                Tweets Country
 0    Bronwyn waiting for symptoms of a blood clot a...      GB
 1     Hubby had a fall, banged his head. If I had t...      GB
 2    __020 As with flu jabs , you have one every ye...      GB
 3      A 16 year old on our unit was sent to hospit...      GB
 4     "extremely rare blood clot" wasn't cutting it...      GB
 ..                                                 ...     ...
 152  I attempted to contact  after I was told they ...      GB
 153    So even at 13.2 per million the risk of bloo...      GB
 154  ...intense headaches from stress, but otherwis...      GB
 155  I hate it that I've had the stupid bloody bloo...      GB
 156   My volunteer consent-taker said that symptoms...      GB
 
 [157 rows x 2 columns],
                                                Tweets Country
 0     _Lawes _ Great to see you safe and well Tom....      GB
 1   Wow-vaccinated two weeks before against a viru...      GB
 2    Given each

In [13]:
merged_df = pd.concat(search_df_list)  # merging all dataframes into one

In [14]:
merged_df = merged_df.reset_index(drop=True)  # resetting rows
merged_df = merged_df.drop_duplicates()  # removing duplicate rows
merged_df = merged_df.reset_index(drop=True)  # resetting rows
merged_df

,Tweets,Country
0,Bronwyn waiting for symptoms of a blood clot a...,GB
1,"Hubby had a fall, banged his head. If I had t...",GB
2,"__020 As with flu jabs , you have one every ye...",GB
3,A 16 year old on our unit was sent to hospit...,GB
4,"""extremely rare blood clot"" wasn't cutting it...",GB
...,...,...
12862,this girl is on PFIZER 🤍,IT
12863,i don’t have a Pfizer.but i have ไฟร่านนน # @...,IT
12864,{283 infiammazioni cardiache su 130milioni di ...,IT
12865,This girl is on Pfizer + uomo Moderna ♥️,IT


# Sentiment Analysis

## NLTK

This section is responsible for the sentiment analysis of each tweet collected. NLTK sentiment analyzer will be used for this project and is achieved by evaluating a sentiment score for each tweet, and then determining the sentiment of that tweet based on its score. The polarity score and the sentiment are 2 separate columns in each dataframe and are then converted to a TSV file for further use.


- NB: In this project, to ensure there are no errors, comment unused code and change the file name of each dataframe stored as TSV prior to convertion.

In [15]:
sia = SentimentIntensityAnalyzer()
calcPolarity = lambda x:sia.polarity_scores(x)['compound']

In [16]:
def sentimentAnalysisNLTK(df):
    df['Compound'] = df['Tweets'].apply(calcPolarity)
    
    return df

In [17]:
df_GB_cleaned = sentimentAnalysisNLTK(df_GB_cleaned)
df_IT_cleaned = sentimentAnalysisNLTK(df_IT_cleaned)
df_ES_cleaned = sentimentAnalysisNLTK(df_ES_cleaned)

merged_df = sentimentAnalysisNLTK(merged_df)

## Determining Sentiment from Polarity Score

In [18]:
# Creating a function that will compute the positive, neutral and negative

def computeAnalysis(polarity):

    if polarity > 0:
        return "Positive"
    elif polarity == 0:
        return "Neutral"
    else:
        return "Negative"

In [19]:
df_GB_cleaned['Analysis'] = df_GB_cleaned['Compound'].apply(computeAnalysis)
df_IT_cleaned['Analysis'] = df_IT_cleaned['Compound'].apply(computeAnalysis)
df_ES_cleaned['Analysis'] = df_ES_cleaned['Compound'].apply(computeAnalysis)

merged_df['Analysis'] = merged_df['Compound'].apply(computeAnalysis)
merged_df

,Tweets,Country,Compound,Analysis
0,Bronwyn waiting for symptoms of a blood clot a...,GB,0.0000,Neutral
1,"Hubby had a fall, banged his head. If I had t...",GB,0.0000,Neutral
2,"__020 As with flu jabs , you have one every ye...",GB,-0.6696,Negative
3,A 16 year old on our unit was sent to hospit...,GB,0.0000,Neutral
4,"""extremely rare blood clot"" wasn't cutting it...",GB,0.0951,Positive
...,...,...,...,...
12862,this girl is on PFIZER 🤍,IT,0.0000,Neutral
12863,i don’t have a Pfizer.but i have ไฟร่านนน # @...,IT,0.0000,Neutral
12864,{283 infiammazioni cardiache su 130milioni di ...,IT,0.0000,Neutral
12865,This girl is on Pfizer + uomo Moderna ♥️,IT,0.0000,Neutral


In [20]:
# Write a dataframe to tsv file without index 

merged_df.to_csv("Searched_Dataset_Cleaned/Searched_Tweets_1.13.tsv", sep="\t", index=False)